# Exploring Palace Partitioning

In [ ]:
import pickle
from pathlib import Path

import graphviz as gv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from gtsam import Pose3

from gtsfm.graph_partitioner.metis_partitioner import MetisPartitioner
from gtsfm.products.visibility_graph import (VisibilityGraph,
                                             visibility_graph_keys)
# from gtsfm.utils.io import read_cameras_txt, read_images_txt
from gtsfm.utils.io import save_poses, load_poses

PALACE = Path("../tests/data/palace")

In [ ]:
sim = np.loadtxt(PALACE / "netvlad_similarity_matrix.txt", delimiter=",")

In [ ]:
plt.imshow(np.triu(sim))
plt.title("Image Similarity Matrix")

> TODO: run MegaLoc and show 

In [ ]:
df = pd.read_csv(PALACE / 'visibility_graph.csv')
graph : VisibilityGraph = list(zip(df["i"], df["j"]))

In [ ]:
print("Number of edges in visibility graph:", len(graph))
print("Number of keys:", len(visibility_graph_keys(graph)))

In [ ]:
# Poses were created with this code but then saved in palace/poses.pkl with save_poses in tiny file
# colmap_path = Path("../results/ba_output")
# poses, img_fnames = read_images_txt(str(colmap_path / "images.txt"))
# save_poses(poses, PALACE / "poses.pkl")
poses = load_poses(PALACE / "poses.pkl")

In [ ]:
xy = np.array([p.translation() for p in poses])

# --- Precompute all edge segments at once ---
valid_edges = [(i, j) for i, j in graph if i < len(xy) and j < len(xy)]
if valid_edges:
    edges_arr = np.array(valid_edges)
    xe = np.empty(3 * len(edges_arr))
    ye = np.empty(3 * len(edges_arr))
    xe[0::3] = xy[edges_arr[:, 0], 0]
    ye[0::3] = xy[edges_arr[:, 0], 1]
    xe[1::3] = xy[edges_arr[:, 1], 0]
    ye[1::3] = xy[edges_arr[:, 1], 1]
    xe[2::3] = np.nan  # separator between segments
    ye[2::3] = np.nan

# --- Build figure with edges first (drawn underneath) ---
fig = go.Figure()

if valid_edges:
    fig.add_trace(
        go.Scatter(
            x=xe,
            y=ye,
            mode="lines",
            line=dict(width=1, color="lightgray"),
            hoverinfo="none",
            showlegend=False,
        )
    )

# --- Add poses as markers (drawn on top) ---
fig.add_trace(go.Scatter(x=xy[:, 0], y=xy[:, 1], mode="markers", marker=dict(size=5)))

fig.update_layout(
    xaxis_title="x",
    yaxis_title="y",
    yaxis_scaleanchor="x",
    yaxis_scaleratio=1,
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()

In [ ]:
partitioner = MetisPartitioner()
cluster_tree = partitioner.run(graph)


In [ ]:
leaves = tuple(cluster_tree.leaves()) if cluster_tree is not None else ()
for index, leaf in enumerate(leaves, 1):
    keys = leaf.local_keys()
    print(f"Leaf {index} has {len(keys)} keys.")
    print(keys)

In [ ]:
cluster_tree

In [ ]:
bayes_tree = partitioner.symbolic_bayes_tree(graph)

In [ ]:
# Comment out to see
# gv.Source(bayes_tree.dot())

In [ ]:
xy = np.array([p.translation() for p in poses])
N = len(xy)

edges_arr = np.asarray(graph, dtype=int)
edges_arr = edges_arr[(edges_arr[:,0] < N) & (edges_arr[:,1] < N)]

fig = go.Figure()

# --- background: all edges, very faint (drawn first, under everything) ---
if edges_arr.size:
    xe_bg = np.empty(3 * len(edges_arr)); ye_bg = np.empty(3 * len(edges_arr))
    xe_bg[0::3] = xy[edges_arr[:,0], 0]; ye_bg[0::3] = xy[edges_arr[:,0], 1]
    xe_bg[1::3] = xy[edges_arr[:,1], 0]; ye_bg[1::3] = xy[edges_arr[:,1], 1]
    xe_bg[2::3] = np.nan;                ye_bg[2::3] = np.nan
    fig.add_trace(go.Scatter(
        x=xe_bg, y=ye_bg, mode="lines",
        line=dict(width=1, color="lightgray"),
        opacity=0.12, hoverinfo="none", showlegend=False
    ))

fig.add_trace(go.Scatter(
    x=xy[:, 0],
    y=xy[:, 1],
    mode="markers",
    marker=dict(size=3, color="lightgray"),
    customdata=np.arange(N),
    hovertemplate="node %{customdata}<extra></extra>",
))

# one legend entry toggles both traces in the same group
fig.update_layout(legend=dict(groupclick="togglegroup"))

for idx, leaf in enumerate(leaves, 1):
    leaf_name = f"Leaf {idx}"
    legendgroup = f"leaf{idx}"

    # node indices (guarded)
    nodes = np.array([k for k in leaf.all_keys() if 0 <= k < N], dtype=int)
    if nodes.size == 0:
        continue

    # edges inside the leaf
    if edges_arr.size:
        m = np.isin(edges_arr[:,0], nodes) & np.isin(edges_arr[:,1], nodes)
        E = edges_arr[m]
        if len(E):
            xe = np.empty(3 * len(E)); ye = np.empty(3 * len(E))
            xe[0::3] = xy[E[:,0], 0]; ye[0::3] = xy[E[:,0], 1]
            xe[1::3] = xy[E[:,1], 0]; ye[1::3] = xy[E[:,1], 1]
            xe[2::3] = np.nan;        ye[2::3] = np.nan
            fig.add_trace(go.Scatter(
                x=xe, y=ye, mode="lines",
                line=dict(width=1), hoverinfo="none",
                name=leaf_name, legendgroup=legendgroup, showlegend=True
            ))

    fig.add_trace(go.Scatter(
        x=xy[nodes, 0],
        y=xy[nodes, 1],
        mode="markers",
        marker=dict(size=6),
        name=leaf_name,
        legendgroup=legendgroup,
        showlegend=False,
        customdata=nodes,  # store node IDs
        hovertemplate="node %{customdata}<extra></extra>",  # clean tooltip
    ))

fig.update_layout(
    xaxis_title=None,
    yaxis_title=None,
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    paper_bgcolor="white",
    plot_bgcolor="white",
    margin=dict(l=0, r=0, t=0, b=0),
    legend=dict(orientation="v", x=0, xanchor="right", y=1, yanchor="top"),
)

# reduce axis padding
fig.update_xaxes(automargin=True)
fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
fig.write_html("visibility_graph.html", include_plotlyjs="cdn", full_html=True)

In [ ]:
!scp visibility_graph.html dellaert@macbook-pro-6:/Users/dellaert/Downloads